In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch



In [25]:
import torch



In [26]:
#file path
file_path = 'test.ft.txt'



In [27]:
# Read the file line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line by space and extract label and text
        label = int(line.split()[0].replace('__label__', ''))
        text = ' '.join(line.split()[1:])
        labels.append(label)
        texts.append(text)

# Convert to DataFrame for easier manipulation
df = pd.DataFrame({
    'label': labels,
    'text': texts
})

# Check the first few rows
print(df.head())

   label                                               text
0      2  Great CD: My lovely Pat has one of the GREAT v...
1      2  One of the best game music soundtracks - for a...
2      1  Batteries died within a year ...: I bought thi...
3      2  works fine, but Maha Energy is better: Check o...
4      2  Great for the non-audiophile: Reviewed quite a...


In [23]:
#train file path
file_pathtrain = 'train.ft.txt'

# Read the file line by line
with open(file_pathtrain, 'r', encoding='utf-8') as file:
    for line in file:
        # Split each line by space and extract label and text
        label = int(line.split()[0].replace('__label__', ''))
        text = ' '.join(line.split()[1:])
        labels.append(label)
        texts.append(text)

# Convert to DataFrame for easier manipulation
dftrain = pd.DataFrame({
    'label': labels,
    'text': texts
})

# Check the first few rows
print(dftrain.head())


   label                                               text
0      2  Great CD: My lovely Pat has one of the GREAT v...
1      2  One of the best game music soundtracks - for a...
2      1  Batteries died within a year ...: I bought thi...
3      2  works fine, but Maha Energy is better: Check o...
4      2  Great for the non-audiophile: Reviewed quite a...


In [28]:
# Preprocess the data
df['label'] = df['label'].map({2: 'positive', 1: 'negative'})  # Example mapping

train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

In [ ]:

# Preprocess the data
df['label'] = df['label'].map({'positive': 2, 'negative': 1})  # Example mapping
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Load the RoBERTa tokenizer and model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', clean_up_tokenization_spaces=True)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenize the input texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Create a dataset class
class AmazonReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = AmazonReviewsDataset(train_encodings, train_labels.tolist())
val_dataset = AmazonReviewsDataset(val_encodings, val_labels.tolist())

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

C:\Users\Candice\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define the compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)  # Get the predicted class by taking the argmax of logits
    labels = p.label_ids  # Ground truth labels
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Update the Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Add this line for evaluation metrics
)

# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)